# Basic I/O

## Outline

* Intro to impure functions

* Basic IO actions
  - Printing messages to the user

  - Retrieving input data from the user

  - Reading from the file system

* Composing IO actions

* The do block
  - Nesting do-blocks

* Recap

## Pure functions

So far, we've been working with pure functions. These functions have no side-efects and (see diagram) take all it's arguments as input and produce a value as an output that depends only on those arguments and the function's logic.

-- TODO: Diagram --

What we mean by input, output, and side-effect is crucial here. A function's input can be only the values we provide as arguments, and a function's output is the value the function returns.

For example:

-- TODO: add 2 or 3 different functions with its inputs and outputs aligned with the Diagram above --

But that begs the question, what if we want to make an interactive program? A website? A game? When we write our program, we have no idea of what the user will do with it. We can't know in advance if a player in our game will move to the left or right, or a user in our website will click in a specific button or not. Those are things that happen while running the program, so there's no way for the programmer to pass them as inputs of a function.

I mean, we could, but imagine if we chose them beforehand. A game that always does the same and finishes in the same way without any way for the player to interact wih it. That's sounds more like a movie, still not bad. Until you realize that you can't even show the image on the screen, cause that would entail sending information and mantaining a connection with your computer's screen, that is outside our program. So, if you run this "game", you're basically using your comptuer as a very expensive heater.

The only way to provide our program with the information and capabilities it needs, is to give it a way to interact with the real world.

And for that, Haskell uses IO actions.

## Introduction to IO Actions

Before starting, with IO actions, I'll address the elephant in the room. I just told you that everything we coded so far was pure and we couldn't interact with it, but we've been running our functions and passing arguments like it was nothing since lesson one! That's because we've been cheeting by using GHCi, that performs IO actions in the background without explicitly telling us. So, at the end of the day, if we want our program to interact with the real world, we still need IO actions.

In Haskell, when we want to do something that has a side effect, we use actions. There are many kinds of actions, but today we'll learn about IO actions, one of the most common and important ones.

The name IO actions gives room for missinterpretation. When we talk about IO actions, we're not talking about the input and output of the function, we talk about the input and output between our program and the real world. IO actions can interact with and change the world outside our program. They might or might not actually have side effects, but they CAN. That's key. They are allowed to have side effects.

***IO action* (or just *action*/*computation*):** Are actions that can interact with and change the world outside our program. 

Let's see a few examples:

- Obtain the result of what a person typed in the keyboard.

- Show some text, an image, or something on the screen

- Save to, or get something from, a data base.

- Call an API (doesn't really matter what you do with the API).

- CRUD a file/directory on a filesystem.

Ok, so, we are clear about what IO actions are. Now, how does Haskell handle IO actions?

## IO actions under the hood

Haskell allows for controled side-effects by explicitly tagging them in the type system.

Now, I'm going to show you the definition of the type that alows us to safely interact with the real world using IO actions. I wasn't going to show you, but I know you'd go behind my back to see it, so I might aswell show it to you. A heads up, don't try to understand the code, we'll only use it to create a mental model of what's happening under the hood.

Without further ado, here's the `IO` type:

```haskell
newtype IO a = IO (State# RealWorld -> (# State# RealWorld, a #))
```

As you can see, `IO` is a type constructor that takes one concrete type as parameter. Now, what hapens after the equal sign looks like magic, and it might aswell be, because that's completely outside the scope of this course. But we can still use it to create a mental model.

There's a single `IO` value constructor. It takes a function that takes the state of the whole real world, does something to it, and returns a tuple containing the state of the real world that may have changed, and a value of type `a` that was generated after whatever the action did.

Aren't the designers of Haskell a bunch of smarty-pants? If you give a function the state of the whole real world, that function can do anything! Talk to databases, allow penguins to fly, fix world-wide catastrophes, anything!. We'll use it to print stuff in the screen, though.

Of course, this is not what is really happening under this seamingly magical type, but is useful to see how we can put together the idea of functions that only operate with the inputs they get, and interacting with the real world. If the real world is the input, the idea we have about how a function works, holds.

Now, let's switch to the practical side. In practice, we don't care about the details of how the interaction with the real world is handled. We don't even care about how the `IO` type is defined! The only thing that we care is that, if we use it properly, the compiler will handle the details and we won't get any surprises when running our code. So let's learn how to properly use the `IO` type.

## IO actions in practice

```haskell
something :: IO a
```

The `IO a` type tells us that `something` is an IO action that, after interacting with the real world, returns a value of type `a`.

For example:

```haskell
action1 :: IO Bool             -- Performs IO action and returns a Bool afterwards

action2 :: IO Int              -- Performs IO action and returns an Int afterwards

action3 :: IO (Double -> Char) -- Performs IO action and returns a function afterwards

action4 :: IO ()               -- Performs IO action and doesn't return anything important
```

There are two key things to note here:
1. One is that, after performing the action, we get a value of the specified type that we can use in our code.
2. The other is that, the action returns a value AFTER performaing the IO action. We CAN NOT get that value without interacting with the real world. You'll see how this affects how we use IO actions.
3. And finally, we see a new type in the last action: The unit type.

### The unit type

```haskell
data () = ()
```

This is the unit type. We see that it has only one value constructor that takes no parameters (also called nullary constructor). Also, like lists, the type and the value look the same. And we use it to represent a value with no information. It's similar but not quite like the "void" value of Java, C, and other languages.

But, wait, why do we learn about this just now? Well, because, untill now, we've been working with pure functions. If the only thing a pure function does is to return `()`, why do you even bother to use that function? Just use the value directly! And if a function takes the unit value as parameter, why do you even bother in requiring that as paramter if it's always the same value? Just remove that parameter and use unit inside the function directly! 

So, when you think about it, the unit type can be completely removed from any pure function and you loose nothing. BUT! Now that we're dealing with actions and side effects, there are plenty of cases when we don't really care about what the action returns as a value but we care only about the side effect it performs. Like printing something on the screen, or deleting a file. We don't need a value in return, we care only about the side effect.

That's why now it makes sense to have this type to represent a value that we don't care about.

If it doesn't quite click, don't worry, we'll see a couple of real-life examples during this lesson. But before that, let's see real life examples of simple actions.

### Retrieving input data from the user

The most basic IO action I can think of is `getChar`:

In [12]:
-- getChar :: IO Char
getChar

: 

This function performs the IO action of asking the user to write a single character on the standard input. And the value of type `Char` it returns is the character the user wrote. We can run the function here to see how that looks.

As you can see, when we run this cell, a text area appears so we can input a character. There's still some magic going on because we're using this Jupyter cells that run GHCi under the hood, but we'll get rid of them soon enough. 


Now, there are cases when a single character is enough, but what about a whole phrase? For that, we can use another function that comes with the base library in Haskell. The `getLine` function:

In [4]:
-- getLine :: IO String
getLine

: 

This one performs the IO action of of asking the user to write a line on the standard input. And the value of type String it returns is the text the user wrote until it pressed Enter.

The `getChar` and `getLine` functions are great and all, but if we run a program with just these, it will prompt us for a character or string without any explanation. Like if you where minding your own business, and all of the sudden, someone stands in front of you with their hand out and silently stares at you. We need a way not only to get but to also send messages to the outside world, and for that, we have to use our first impure function.

### Actions VS Impure Functions  (printing messages to the user)

Let me present you the `putStrLn` function:

```haskell
putStrLn :: String -> IO ()
```

`putStrLn` takes a string and returns an IO action that, after it's performed, returns a unit value, a value we don't care about. It has the `IO` type constructor in its type, but it is not an action. It's a function that takes one argument (a string) and returns an action of type `IO ()`. The distinction is subtle but important. IO actions will never require additional arguments, but we can have functions that perform actions (such as putStrLn). And those are what we call impure functions.

So, what does the action of `putStrLn` do? It takes the string we pass as parameter and prints it in the standard output:

In [1]:
putStrLn "Hello World from inside the program!"

Hello World from inside the program!

-- Diagram

```haskell
(putStrLn)      :: String -> IO ()

(putStrLn "Hi") :: IO ()
```

What's happenning here, is that `putStrLn` is applied to the String and it returns an action of type IO (). Then, because we're using a Jupyter notebook that automatically performs the action of the top function and prints the result, we get the String in our standard output, below the cell. 

But check this out:

In [6]:
x = putStrLn "Hello World from inside the program!"

:t x

x :: IO ()

As you can see, if we run that cell, we don't get the String in our standard output. That's because we didn't ask for the action to be performed. We just named it `x` and that's it. We never needed it, so we didn't perform it.

And that's another property of actions, they are what in programming is called first-class values.

## Actions are first-class values

This means that you can treat actions the same as any other value. For example:

In [32]:
-- putStrLn is an example of a function that returns an action


-- Bind to names
x = putStrLn "Hello World from inside the program!"


-- Put them inside lists or other data structures
listOfActions :: [IO ()]
listOfActions = [putStrLn "a", putStrLn "b"]


-- Pass them as function parameters
fakeLength :: [IO ()] -> Int
fakeLength list = 1 + length listOfActions

And, like it happened before, if we run the `fakeLenghth` function passing the `listOfActions` as parameters...

In [33]:
fakeLength listOfActions

3

We don't get any of the messages in the standard ouput because none of the actions inside the list is performed! Why should they be performed? We just asked about the length of the list, not the values it contains. Remember that Haskell is lazier than a cat under the sun. It won't do anything unles it has to. 

So, how can we ask Haskell to perform a few different actions? By composing them with special operators.

## Composing IO actions (`>>` and `>>=` operators)

To learn how to combine IO actions, we'll create a few bots.

### Rude bot

The first one is a rude bot. As soon as you interact with it, it yells at you.

We'll start with a single meesage:

In [8]:
rudeBot :: IO ()
rudeBot = putStrLn "Hey!"

rudeBot

Hey!

The bot has to catch it's breath before yelling at you again, so we'll add the next phrase in a second action. To do that, we'll introduce the "then" operator:

```haskell
(>>) :: IO a -> IO b -> IO b
```

As you can see, this operator takes two actions IO a and IO b. It first executes IO a, ignores the result (a), and returns the IO b action.

If this where a pure operator, the implementation would look like this:

In [28]:
pureThen :: a -> b -> b
x `pureThen` y = y


3 `pureThen` 5

5

We'd be just throwing away the first value and returing the second.

But, because we're dealing with IO actions, this operator has some secret sauce that allows it to perform the first IO action before returning the second one. More specifically:

The `>>` operator sequentially composes two actions, discarding any value produced by the first.

For example:

In [35]:
abc = putStrLn "a" >> putStrLn "b" >> putStrLn "c"

abc

a
b
c

The direction of the arrows of the operator tells us the direction of the sequence, so, we perform actions from left to right.

Now, le'ts use this operator to finish the rude bot:

In [36]:
rudeBot :: IO ()
rudeBot = putStrLn "Hey!" >> putStrLn "Get out of my lawn!"

rudeBot

Hey!
Get out of my lawn!

Exactly what we wanted!

But I think we could make it even ruder.

### An even ruder bot

How could we make this bot more anoying? Hmm.. I know! Let's make it seem interested and then yell at us! So it also wastes our time.

So, we'll make the bot ask our name, completely ignore it, and then yell at us. All that with this simple code:

In [37]:
evenRuderBot :: IO ()
evenRuderBot =
  putStrLn "What's your name?"            -- IO ()
    >> getLine                            -- IO String
    >> putStrLn "like I care! Get lost!"  -- IO ()
    
evenRuderBot

: 

It's getting a bit too long, so we'll split it into several lines.

Besides that, not much has changed. The only real change is that we added a `getLine` function inbetween the `putStrLn` ones. But if we run it now, we see that:

1. First, it performs the side effect of asking for our name in the standard output.
2. THEN, it waits for us to type it.
3. And THEN, after that side effect finishes, it sends the final message.

We can see ins the types that the second action returned a string. But because we used the `>>` operator, we completely ignore it. Let's learn how to actually make use of this value by taking our rude bot to therapy.

### Rude bot  after theraphy

After going to therapy, our bot feels way better and now wants to be friends with us. And for that, it has to use our name. We can't use the `>>` operator, cause that ignores it. So, we need another operator that does the same but that passes the result of the first action to the second one. And for that, we use the "bind" operator:

```haskell
(>>=) :: IO a -> (a -> IO b) -> IO b
```

It looks complicated, but it's not that different from the `>>` operator.

This operator takes and action `IO a` and an impure function that takes a value of the same type `a` as the one produced by the first action and returns another action IO b. 

So, what this operator does is to execute `IO a` to get the value `a`, and then pass that value to the impure function to get the `IO b` action as result. 

If this where a pure operator, the implementation would look like this:

In [38]:
pureBind :: a -> (a -> b) -> b
x `pureBind` f = f x


3 `pureBind` (5+)  -- Same as just doing (5+3)

8

So it would be just taking a value and a function and aplying the function to the value. Pretty useless.

But because we're dealing with side effects, we don't actually have the value `a` until we perform the action, so that's what this operator does:

The `>>=` operator sequentially composes two actions, passing any value produced by the first as an argument to the second.

For example:

In [42]:
-- Remainders:
-- (>>=) :: IO a  -> (a -> IO b)  -> IO b
-- getLine :: IO String


yellIt :: String -> IO ()
yellIt str = putStrLn (str ++ "!!!!!!")

yellItBack :: IO ()
yellItBack = getLine >>= yellIt

yellIt "Hey"
--yellItBack

Hey!!!!!!

As you can see, we have the `yellIt` function that takes a String and returns the action of printing that string with a bunch of exclamation marks. And in the `yellItBack` function, we perfor the acton of `getLine` that returns a String. But instead of throwing the string away, this time we  use the `>>=` operator to pass it as the paramerer of `yellIt`. Yell it returns that action, and because it's the last one, it gets evaluated too. Sending the outside world the same message but with more emphasis.

Now, le'ts use this operator on the—now nicer—bot:

In [45]:
rudeBotAfterTherapy :: IO ()
rudeBotAfterTherapy =
  putStrLn "What's your name?"
    >> getLine
    >>= (\name -> putStrLn $ "Nice to meet you, " ++ name ++ "!")
    

rudeBotAfterTherapy

: 

As you can see, instead of defining a function and use it inside the bot, chose to use a lambda function. Why? Because I'm using this function only once. And creating a sepparate function, having to think about a name for it, and polluting the environment with a function I know I'll never use again, it's not a good idea. In those cases, we prefer lambda functions.

So, now that ....*****

### Now it's a chatty bot!

Our bot is now fully ~~recovered~~ and, it turns out that it's a chatty bot! Let's add a few chatty features! 

For example, let's add another message that tells us the number of letters our name has:

In [46]:
lettersInName :: String -> String
lettersInName name =
  "Your name has "
    ++ show (length name)
    ++ " letters, in case you where wandering..."

The `lettersInName` function is a pure function that takes a name and does a silly little comment on it. To add it to our chatty bot, we need to do it like this:

In [47]:
chattyBot :: IO ()
chattyBot =
  putStrLn "Hey! What's your name?"
    >> getLine
    >>= ( \name ->
            putStrLn ("Nice to meet you, " ++ name ++ "!")
              >> putStrLn (lettersInName name)
        )

We need the value `name` (provided as result of the second action) again, so we need to keep building our function inside that lambda function.

Now it's more of the same, we can keep adding more and more actions like that. This is something a bit more complicated:

In [48]:
finalChattyBot :: IO ()
finalChattyBot =
  putStrLn "Hey! What's your name?"
    >> getLine
    >>= ( \name ->
            putStrLn ("Nice to meet you, " ++ name ++ "!")
              >> putStrLn (lettersInName name)
              >> putStrLn ("So, " ++ name ++ ", what do you do for fun?")
              >> getLine
              >>= ( \hobby ->
                      putStrLn ("Are you kidding, " ++ name ++ "! I love " ++ hobby ++ "!")
                  )
        )
    >> putStrLn "OK, bye!"

As you can see, if we keep increasing the interactions, we start to see a pattern. An uggly and hard to read pattern.

Luckily, we're not the first ones to encounter this monstruosity, and the haskell gods did something about it. Enter, the "Do Notation"

## The `do` notation

The `do` notation is just syntactic sugar for expressions composed by `>>` an `>>=` operators. Basically, a nicer way of combining actions.

We're going to rewrite all the previous expression with do notation so we can see the difference. Starting with the rude bot:

In [52]:
rudeBot :: IO ()
rudeBot = putStrLn "Hey!"


rudeBotDo :: IO ()
rudeBotDo = do
    putStrLn "Hey!"

As you can see, we write the keyword `do` after the equal sign, and then start a block with the actions. This one isn't particularly special. It's even more clear and sucinct to write it without do syntax!

Let's see the the second version of rude bot:

In [53]:
rudeBot :: IO ()
rudeBot = putStrLn "Hey!" >> putStrLn "Get out of my lawn!"

rudeBotDo :: IO ()
rudeBotDo = do 
    putStrLn "Hey!"
    putStrLn "Get out of my lawn!"

rudeBotDo

Hey!
Get out of my lawn!

Now we start to see some improvement. It's not much, but we can see that each action is in a different line, making it easier to identify them. Before, the actions went from left to right, now, from top to bottom. 

And what about the even ruder bot?

In [ ]:
evenRuderBot :: IO ()
evenRuderBot =
  putStrLn "What's your name?"  
    >> getLine  
    >> putStrLn "like I care! Get lost!"
    
    
evenRuderBotDo :: IO ()
evenRuderBotDo = do
  putStrLn "What's your name?"  
  getLine  
  putStrLn "like I care! Get lost!"
    
evenRuderBotDo

: 

Same as before, add the `do` keyword, remove the `>>` operators, and you're ready to go.

Now is when the cool stuff starts. We have the rude bot after therapy:

In [ ]:
rudeBotAfterTherapy :: IO ()
rudeBotAfterTherapy =
  putStrLn "What's your name?"
    >> getLine
    >>= (\name -> putStrLn $ "Nice to meet you, " ++ name ++ "!")

How do we handle the `name`? For that, we'll introduce the `<-` (left arrow, or bind):

In [ ]:
rudeBotAfterTherapyDo :: IO ()
rudeBotAfterTherapyDo = do
  putStrLn "What's your name?"
  name <- getLine -- (getline :: IO String) so (name :: Sring)
  putStrLn $ "Nice to meet you, " ++ name

This left arrow binds the result of running the `putStrLn` action to `name`. And once you have the `name` variable, you can use it anywhere after that action. 

Now let's see how we can use do notation for chatty bot:

In [ ]:
chattyBot :: IO ()
chattyBot =
  putStrLn "Hey! What's your name?"
    >> getLine
    >>= ( \name ->
            putStrLn ("Nice to meet you, " ++ name ++ "!")
              >> putStrLn (lettersInName name)
        )


chattyBotDo :: IO ()
chattyBotDo = do
  putStrLn "Hey! What's your name?"
  name <- getLine
  putStrLn ("Nice to meet you, " ++ name ++ "!")
  putStrLn $ lettersInName name

Now the differences starts to get big! You have to stop on `chattyBot` for a few seconds to grasp what it's doing, but `chattyBotDo` is really easy to follow! 

Finally, let's compare the most complicated one:

In [ ]:
finalChattyBot :: IO ()
finalChattyBot =
  putStrLn "Hey! What's your name?"
    >> getLine
    >>= ( \name ->
            putStrLn ("Nice to meet you, " ++ name ++ "!")
              >> putStrLn (lettersInName name)
              >> putStrLn ("So, " ++ name ++ ", what do you do for fun?")
              >> getLine
              >>= ( \hobby ->
                      putStrLn ("Are you kidding, " ++ name ++ "! I love " ++ hobby ++ "!")
                  )
        )
    >> putStrLn "OK, bye!"

In [ ]:
finalChattyBotDo :: IO ()
finalChattyBotDo = do
  putStrLn "Hey! What's your name?"
  name <- getLine
  putStrLn ("Nice to meet you, " ++ name ++ "!")
  putStrLn (lettersInName name)
  putStrLn ("So, " ++ name ++ ", what do you do for fun?")
  hobby <- getLine
  putStrLn ("Are you kidding, " ++ name ++ "! I love " ++ hobby ++ "!")
  putStrLn "OK, bye!"
  
  
finalChattyBot

Notice that all the statemetns are aligned at the same indentation. Make sure to always keep them aligned if they belong to the same do block, to avoid confusing the compiler and having silly errors.

As you can see, do notation is great! It allows for a more clean and concise code. But... waaaaait a minute! This code looks... really imperative! You're stating what to do step by step in sequence!

This looks oddly non-declarative! What happened? Wasn't it that declarative code was so great? Did I lie to you when I compared declarative vs ...? There's a reason why we now use a series of statements instead of describing what we want. And that's because, now, we really care about what happens and in which order! Because side effects can cause all sorts of troubles!!! For example: functions with descriptive names in sequence that, if rearranged, could cause caos

That's the do notation in essence. But there are a few other things that cover the practical aspect of using do notation that I'd like to cover. We'll drop the chatbot example cause it's starting to get borring.

Let's start with using `let` inside the `do` notation:

### using `let` inside the `do` notation

We can use the `let` keyword inside a `do` block like this:

In [ ]:
code

There are a few details to keep in mind:
- The constructs binded with the `let` keyword are lazy. So, even though they occupy a line in the do block, they aren't evaluated unless needed somewhere else.
- We don't need the `in` kewowrd like we do outside `do` notation. It's assumed that ..

### Nesting `do` blocks

We can nest do blocks as much as we want. For example:

In [ ]:
code

### Escaping `IO` and the `return` keyword

**`return` DOES NOT WORK LIKE IN OTHER PROGRAMMING LANGUAGES! ITS A DIFFERENT THING!**

If you don't konw any other programming languages, don't worry about it. But you likely do, so I wanted to open with that to avoid confusion or in case you where about to skip this section.

In [ ]:
joinStrings :: IO String
joinStrings = do
  putStrLn "Please, provide your name"
  n <- getLine
  putStrLn "Please, provide your lastname"
  ln <- getLine
  return (n ++ ln)

-------------------------------------------------------------------------------

-------------------------------------------------------------------------------

-------------------------------------------------------------------------------

-------------------------------------------------------------------------------

In [ ]:
-- A pure function that prepends the string "Hello " to another string
addHello :: String -> String
addHello = ("Hello " ++)

-- Applying the putStrLn function to an expression that evaluates to a String
putStrLn (addHello "John")

Here, `putStrLn` needs the result of evaluating the expression inside the paranthesis, so Haskell applies `addHello` to `"John"` and then applies `putStrLn` to the result.  

Now that we can use actions to do useful side effects outside our program, we would like to be able to compose two IO actions.

We could try to use another `IO a` as an input to get a function of type `IO a -> IO b`. 

In [ ]:
-- An IO action that perform the action of printing "Hello". 
someIOType :: IO ()
someIOType = putStrLn "Hello"

-- A naive and wrong function that can combine two IO () actions
combineIO :: IO () -> IO ()
combineIO io = putStrLn "world"

combineIO someIOType

This program works, but it does not print the *Hello* string. The `someIOType` action is not being performed.

This is because the input `IO ()` is not being used in the body of the function, so it is not being evaluated. 

But how can we call this IO action on the right side? To solve this, we introduce two new operators: 
- the **then** or also called **sequence** operator given by `>>`

- the **bind** operator given by `>>=`

These operators allow for the composition of IO actions. Their properties are: 

| `>>` | `>>=` |
| --- | --- |
| Composes two IO actions | Composes two IO actions |
| Does not forward the return type | Forwards the return type |

We first look at an example of how the `>>` operator works.

In [ ]:
-- A new expression that is of type IO ()
printHello :: IO ()
printHello = putStrLn "Hello " 

-- A new expression that is of type IO ()
printWorld :: IO ()
printWorld = putStrLn "World"

--Combining the above IO actions
chaining :: IO ()
chaining = printHello >> printWorld

chaining

Here, the `printHello` first performs some side effect and then the `printWorld` action performs its side effect. 

The arrows of the operator `>>` indicate the flow of the action that is in which order they are performed. 

We could say the type signature for the sequence operator is:
```haskell
(>>) :: IO a -> IO b -> IO b
```
It takes two IO actions, performs both, and returns the result of the second one.

But the actual type signature is:
```haskell
(>>) :: m a -> m b -> m b
```
where `m` has an instance of the monad type class, which `IO` has. We will talk about this type class in lesson 20.

Next, we look at an example of the `>>=` operator works.

In [ ]:
printHelloName :: String -> IO ()
printHelloName name = putStrLn ("Hello " ++ name)

chainingWithBind :: IO ()
chainingWithBind = getLine >>= printHelloName

We see how we get the input string with the `getLine` function and then pass it on to the `printHelloName` function.

The important thing here is that the bind operator removes the `IO` context from the `IO String` variable and passes on only a variable of type `String` forward.

Here we could say the type signature for the bind operator is:
```haskell
(>>=) :: IO a -> (a -> IO b) -> IO b
```
It takes in an IO action that it performs and hands the result without the IO context to a function that produces another IO action.

The actual type signature is:
```haskell
(>>=) :: m a -> (a -> m b) -> m b
```
and also here `m` has an instance of the monad type class.

Another example of the bind operator can be seen below.

In [ ]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
getFiles :: IO [FilePath]
getFiles = listDirectory "." 

-- A new expression that is of type IO ()
printFirstFile :: [FilePath] -> IO ()
printFirstFile = print . head 

--Combining the above IO actions with forwarding the
chainingWithBind :: IO ()
chainingWithBind = getFiles >>= printFirstFile

chainingWithBind

Here we used `print` IO action instead of `putStrLn` to show the first element of the list of file paths. 
```haskell
print :: Show a => a -> IO ()
```
This action automatically converts a type to a string and sends it to StOut, which can save us writing syntax.

### Reading from the file system

The last example we will show here is the `listDirectory` which is located in the module `System.Directory`. 
```haskell
listDirectory :: FilePath -> IO [FilePath]

type FilePath = String
```
It takes in a string that represents the file path to a folder and returns an IO list of strings that are the files and folders contained in the input folder. 

Modules are bundles of Haskell functions that are not available by default unless we import them. We will talk more about this in lesson 12.

Here we show a simple example of how to import this function from its module and use it. 

In [ ]:
import System.Directory (listDirectory)

-- The "." input refers to the current jupyter working directory (week02)
listedDirectories :: IO [String]
listedDirectories = listDirectory "."

listedDirectories

## The do block

Now that we know how to compose actions, we are going to generalize this with the so called **do block**. 

To introduce the necessity and reasons behind this useful syntax, we look at a few examples. 

First, consider the composition of many actions via the then operator `>>`.

In [ ]:
--  IO ()              IO String           IO ()
putStrLn "Action one" >> getLine >> putStrLn "Action three"

Here each action is **sequentially** performed, the first and last concatenated actions print “Action one” and “Action three” respectively. 

The middle action asks for input, which performs no output to the console at all.

Let's look at how this pattern would work for the bind operator `>>=`.

In [ ]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
getFiles :: IO [FilePath]
getFiles = listDirectory "." 

-- A new expression that is of type IO ()
printFirstFile :: [FilePath] -> IO ()
printFirstFile = print . head 

-- A new expression that is of type IO ()
printLastFile :: [FilePath] -> IO ()
printLastFile = print . last 

longerChain :: IO ()
longerChain = getFiles >>= printFirstFile >>= printLastFile

longerChain

The compiler gives an error since we are incorrectly matching the types of the functions used! 

The function `printFirstFile` returns a `()` while the `printLastFile` function expects a `[FilePath]`.

Somehow we need to manage how inputs and outputs of these functions are used. 

Ideally, we would like the `printLastFile` function to use the output of the `getFiles` function. 

Achieving this can be done with lambda functions, that we saw in lesson 5. As a small recap, they take the form

In [ ]:
(\x -> 2*x + 1) 1 -- Here the lambda function (\x -> 2*x + 1) is applied to the argument 1

Using this, we can combine the action while making the inputs **progressively** available to the other actions inline. 

We do this by adding the actions in order to a lambda function. The following example achieves what we wanted:

In [ ]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
getFiles :: IO [FilePath]
getFiles = listDirectory "." 

-- A new expression that is of type IO ()
printFirstFile :: [FilePath] -> IO ()
printFirstFile = print . head 

-- A new expression that is of type IO ()
printLastFile :: [FilePath] -> IO ()
printLastFile = print . last 

chainingWithLambda :: IO ()
chainingWithLambda = getFiles >>= (\x1 -> printFirstFile x1 >> printLastFile x1)

chainingWithLambda

Here we first forward the output of the `grabFiles` action denoted by `x1` into a lambda function. 

Then in the lambda function, we call `printFirstFile` and `printLastFile` and chain them together with `>>`.

Notice that with this construction, all inputs become available to functions that need to use them. 

Now, clearly this construction is very cumbersome and not practical in real life coding!

To make it easier this whole construction can be done by a **do block** which is just syntactic sugar for code that uses the previous approach. 

The do-block looks just like an imperative way of programming, but in its core it is still functional by using lambda functions.  

Let's re-write the previous main function in a do block:

In [ ]:
-- The introduction of the do block
usingDoBlock :: IO ()
usingDoBlock = do
   x1 <- getFiles
   printFirstFile x1
   printLastFile x1

usingDoBlock

The do-block always starts with the `do` keyword. In a do-block, each line can perform some action. 

First we perform the `getFiles` action. After that its output is stored in the variable `x1` using the syntactic arrow `<-`. 

The `x1` variable is now of type `String` because in the background the `>>=` operator is being used. 

Then the next action `printFirstFile` is performed, that takes `x1` as it's input. 

Lastly, the action `printLastFile` is called, that also takes `x1` as it's input.

This do block  is fundamentally the same as the previous example that used a lambda function. 

With these do blocks, we can write clearly structured code that performs multiple side effect. 

This looks oddly non-declarative! What happened? Wasn't it that declarative code was so great? Did I lie to you when I said that declarative code is awesome?

There's a reason why we now use a series of statements instead of describing what we want. And that's because, now, we care about what happens and in which order, because side effects can cause all sorts of troubles!!! For example: *functions with descriptive names in sequence that, if rearranged, could cause caos*

Inside a do block, you can also define let-statements that define variables in the block.

In [ ]:
usingDoBlock :: IO ()
usingDoBlock = do fileList <- getFiles
                  let elemOne = fileList !! 1 -- take the 2nd elemet from the list
                  print elemOne

usingDoBlock 

For the `let` keyword, lambda functions are used in the de-sugared code:

In [ ]:
desured :: IO ()
desured = getFiles >>= (\x1 -> (\x2 -> print x2) (x1 !! 1))

desured

The `let` keyword allows you also to define functions beside variables. The previous example can be re-written to:

In [ ]:
letWithFunction :: IO ()
letWithFunction = do fileList <- getFiles
                     let elemOne xs = xs !! 1 -- elemOne is now a function
                     print $ elemOne fileList

letWithFunction 

### Nesting do-blocks

It may not appear obvious, but do-blocks can be nested. Here is an example:

In [ ]:
import Data.Char ( isDigit )

nesting :: IO ()
nesting = do
    putStrLn "What is your age:"
    ageString <- getLine
    let validAge = all isDigit ageString
    if validAge
    then do
        let age = read ageString :: Int
            msg = "Int 10 years you will be " ++ show (age + 10) ++ " years old."
        putStrLn msg 
    else do
        putStrLn "Your age should contain only digits from 0-9."
        nesting

We import the `isDigit` function that checks whether a character is a number or not.
```haskell
isDigit :: Char -> Bool
```

Then we ask the user to input his age and after that define an if-statement which contains two different do-blocks.

If the input string contains only numbers, we print a message how much will be the user's age in 10 years.

If it does not contain only numbers, then we notify the user and start the program from the beginning. 

Also you may notice in the `then` do-block we define two variable but use the `let` keyword only once.

This is allowed in Haskell if the variable names have the same indentation and follow one after each other. 

Then you need to write `let` only for the first variable.

## Recap

In this section, we have introduced the concepts of IO actions. 

- Haskell knows two types of functions, pure and impure function, where the latter are called IO actions.

- The types of impure functions are wrapped and marked by `IO` and which indicates that the functions may perform useful side effects. 

- Once you use a function or variable type that is wrapped in IO, the function inside which you use it has to be wrapped in IO. 

- IO actions can be composed using two operators, the then or sequence operator denoted by `>>` and the bind operator denoted by `>>=`.

- The do-blocks let you easily compose multiple actions and they can be nested.

- TODO: Why? Why this way and not another? Laziness VS side-effects.

- TODO: What happens with all those advantages we had with pure functions?

## Intro to impure functions

Functions with side effects are called **impure fuunctions**. As a recap:

| Pure | Impure |
| --- | --- |
| Always produces the same result when given the same parameters | May produce different results for the same parameters |
| Never has side effect | May have side effects |
| Never alters state | May alter the global state of the program, system, or world |
| Doesn't have `IO` anywhere | Has `IO` somewhwere |

Haskell strictly separates pure code from the impure interactions with things outside the program. 

Because we would still like our Haskell programs to alter the world outside our program, we can do this via **IO actions** denoted by `IO a`.

Because also **IO actions** can take in arguments as pure functions, we sometimes call them **IO functions** or **impure functions**. 

But strictly speaking, all functions in Haskell are pure functions and the rest are IO actions.

All these impure functions include a type that is marked with the `IO` wrapper followed by a type parameter `a`. 

This tells us that the impure function may first perform some computation with possible side effects and then return something of type `a` wrapped in the `IO` context.

Inside IO actions, it is possible to call pure functions and/or other IO actions. But you can never call an IO action inside a pure function.